# Analise Exploratória de Dados - Despesa Pública

Este notebook tem como objetivo realizar a conexão com o banco de dados e executar as queries de investigação definidas para o desafio.

## 1. Configuração e Conexão

In [1]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Adicionar diretório raiz ao path para importar src
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.database import get_db_engine, run_query, load_query

# Configurar estilo dos gráficos
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

In [2]:
# Testar conexão
engine = get_db_engine()
if engine:
    print("Conexão estabelecida com sucesso!")
else:
    print("Falha na conexão.")

Conexão estabelecida com sucesso!


## 2. Visão Geral dos Dados
Vamos verificar a volumetria das tabelas principais.

In [3]:
query_volumetria = load_query('queries_base/volumetria_geral.sql')

df_volumetria = run_query(query_volumetria)
df_volumetria

,tabela,total
0,contrato,500
1,empenho,497
2,liquidacao_nota_fiscal,457
3,pagamento,497


## 3. Investigação de Anomalias (Questão 2)
Abaixo executamos as queries identificadas para detectar possíveis inconsistências.

### 3.1 Contratos sem Empenho
Contratos que não possuem nenhum empenho associado podem indicar cadastros orfãos ou falta de execução.

In [4]:
query_contratos_sem_empenho = load_query('queries_base/contratos_sem_empenho.sql')

df_contratos_sem_empenho = run_query(query_contratos_sem_empenho)
print(f"Contratos sem empenho: {len(df_contratos_sem_empenho)}")
df_contratos_sem_empenho.head()

Contratos sem empenho: 3


,id_contrato,objeto,data,valor
0,1498,Fornecimento de Papel A4,2024-01-03,15188.58
1,1499,Merenda Escolar - Lote 1,2024-01-03,39185.84
2,1500,Consultoria Financeira,2024-01-10,7046.77


### 3.2 Pagamentos > Valor Empenhado
Situação crítica onde o valor pago supera o que foi reservado (empenhado).

In [6]:
query_pagamentos_maior_empenho = load_query('queries_base/pagamentos_maior_empenho.sql')

df_pagamentos_maior_empenho = run_query(query_pagamentos_maior_empenho)
print(f"Empenhos com pagamento superior ao valor: {len(df_pagamentos_maior_empenho)}")
df_pagamentos_maior_empenho.head()

Empenhos com pagamento superior ao valor: 225


,id_empenho,valor_empenhado,total_pago
0,EMP-2,19621.16,26021.35
1,EMP-5,4993.13,30930.82
2,EMP-13,31200.76,49985.61
3,EMP-18,4553.19,18609.40
4,EMP-19,17109.77,17427.94


### 3.3 Pagamento anterior ao Empenho
A legislação exige que o empenho preceda o pagamento.

In [7]:
query_datas_invalidas = load_query('queries_base/datas_invalidas.sql')

df_datas_invalidas = run_query(query_datas_invalidas)
print(f"Pagamentos realizados antes do empenho: {len(df_datas_invalidas)}")
df_datas_invalidas.head()

Pagamentos realizados antes do empenho: 41


,id_empenho,id_contrato,data_empenho,data_pagamento
0,EMP-4,1333,2024-01-11,2024-01-06
1,EMP-6,1455,2024-01-05,2023-12-31
2,EMP-10,1002,2024-01-13,2024-01-08
3,EMP-14,1045,2024-01-10,2024-01-05
4,EMP-19,1100,2024-01-05,2023-12-31


### 3.4 Pagamentos sem Nota Fiscal
Verifica integridade entre pagamento e liquidação.

In [8]:
query_sem_nf = load_query('queries_base/pagamentos_sem_nf.sql')

df_sem_nf = run_query(query_sem_nf)
print(f"Pagamentos sem nota fiscal associada: {len(df_sem_nf)}")
df_sem_nf.head()

Pagamentos sem nota fiscal associada: 40


,id_pagamento,id_empenho,valor,datapagamentoempenho
0,PGT-28,EMP-28,4654.45,2024-01-24
1,PGT-78,EMP-78,14239.61,2024-02-06
2,PGT-81,EMP-81,26390.18,2024-02-01
3,PGT-107,EMP-107,1516.92,2024-01-21
4,PGT-130,EMP-130,25843.98,2024-02-02


## 4. Próximos Passos
- Aprofundar a análise nos casos encontrados.
- Gerar gráficos para investigar sazonalidade ou concentrações.